In [1]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen

#AWS Test

In [2]:
URL = 'https://fbref.com/en/comps/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
# Big 5 European Leagues (Spain, England, Germany, France, Italy)

big_5_leagues = []

for j in soup.find_all('tbody')[2].find_all("tr", {"class": "gender-m"}):
    if (j.find('td') != None):
        big_5_leagues.append(j.find('a')['href'])

big_5_leagues = big_5_leagues[:-1]

big_5_leagues

['/en/comps/9/history/Premier-League-Seasons',
 '/en/comps/12/history/La-Liga-Seasons',
 '/en/comps/13/history/Ligue-1-Seasons',
 '/en/comps/20/history/Bundesliga-Seasons',
 '/en/comps/11/history/Serie-A-Seasons']

In [5]:
# function to obtain league/season URLs

def get_all_seasons(url):
    URL = 'https://fbref.com/' + url
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    url_list = []
    
    for row in soup.find_all('tr'):
        if row.find('th',{"scope":"row"}) != None:
            url_list.append((row.find('a')['href']))
    
    return url_list

In [6]:
# All Seasons Big 5 Leagues

all_seasons_big_5 = []

for i in big_5_leagues:
    league_seasons = get_all_seasons(i)
    all_seasons_big_5 += league_seasons

all_seasons_big_5

['/en/comps/9/Premier-League-Stats',
 '/en/comps/9/10728/2020-2021-Premier-League-Stats',
 '/en/comps/9/3232/2019-2020-Premier-League-Stats',
 '/en/comps/9/1889/2018-2019-Premier-League-Stats',
 '/en/comps/9/1631/2017-2018-Premier-League-Stats',
 '/en/comps/9/1526/2016-2017-Premier-League-Stats',
 '/en/comps/9/1467/2015-2016-Premier-League-Stats',
 '/en/comps/9/733/2014-2015-Premier-League-Stats',
 '/en/comps/9/669/2013-2014-Premier-League-Stats',
 '/en/comps/9/602/2012-2013-Premier-League-Stats',
 '/en/comps/9/534/2011-2012-Premier-League-Stats',
 '/en/comps/9/467/2010-2011-Premier-League-Stats',
 '/en/comps/9/400/2009-2010-Premier-League-Stats',
 '/en/comps/9/338/2008-2009-Premier-League-Stats',
 '/en/comps/9/282/2007-2008-Premier-League-Stats',
 '/en/comps/9/229/2006-2007-Premier-League-Stats',
 '/en/comps/9/183/2005-2006-Premier-League-Stats',
 '/en/comps/9/146/2004-2005-Premier-League-Stats',
 '/en/comps/9/112/2003-2004-Premier-League-Stats',
 '/en/comps/9/84/2002-2003-Premier-Lea

In [7]:
# print(len(all_seasons))
print(len(all_seasons_big_5))

159


In [8]:
def get_all_player_urls(url_list):
    
    final_player_urls = []

    for i in url_list:
        final_pos = i.rindex('/')

        final_player_urls.append(i[:final_pos+1]+'stats/'+i[final_pos+1:])

    return final_player_urls

In [9]:
# Getting final_stats for all players in the Big 5 Leagues

final_stats_big_5 = get_all_player_urls(all_seasons_big_5)

In [10]:
final_stats_big_5

['/en/comps/9/stats/Premier-League-Stats',
 '/en/comps/9/10728/stats/2020-2021-Premier-League-Stats',
 '/en/comps/9/3232/stats/2019-2020-Premier-League-Stats',
 '/en/comps/9/1889/stats/2018-2019-Premier-League-Stats',
 '/en/comps/9/1631/stats/2017-2018-Premier-League-Stats',
 '/en/comps/9/1526/stats/2016-2017-Premier-League-Stats',
 '/en/comps/9/1467/stats/2015-2016-Premier-League-Stats',
 '/en/comps/9/733/stats/2014-2015-Premier-League-Stats',
 '/en/comps/9/669/stats/2013-2014-Premier-League-Stats',
 '/en/comps/9/602/stats/2012-2013-Premier-League-Stats',
 '/en/comps/9/534/stats/2011-2012-Premier-League-Stats',
 '/en/comps/9/467/stats/2010-2011-Premier-League-Stats',
 '/en/comps/9/400/stats/2009-2010-Premier-League-Stats',
 '/en/comps/9/338/stats/2008-2009-Premier-League-Stats',
 '/en/comps/9/282/stats/2007-2008-Premier-League-Stats',
 '/en/comps/9/229/stats/2006-2007-Premier-League-Stats',
 '/en/comps/9/183/stats/2005-2006-Premier-League-Stats',
 '/en/comps/9/146/stats/2004-2005-Prem

In [11]:
def get_player_table(url):
    url = 'https://fbref.com/' + url
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    player_table = all_tables[2]

    return player_table

In [12]:
def get_player_urls(url_list):
    
    player_url_list = []

    for url in url_list:
        url = 'https://fbref.com/' + url
        res = requests.get(url)
        ## The next two lines get around the issue with comments breaking the parsing.
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",res.text),'lxml')
        all_tables = soup.findAll("tbody")
        try:
            player_table = all_tables[2]
        except:
            next

        for row in player_table.find_all('a'):
            if "matchlogs" in row['href']:
                player_url_list.append(row['href'])

    return player_url_list

In [180]:
stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]

In [13]:
def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)

    return df_player

In [14]:
#Getting player tables for big 5 leagues

player_table_big_5 = get_player_urls(final_stats_big_5)

len(player_table_big_5)

81267

In [162]:
# # Saving player_table as pickle file
# import pickle

# with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_table.pickle', 'wb') as handle:
#     pickle.dump(player_table, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [163]:
# Saving player_table_big_5 as pickle file
import pickle

with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_table_big_5.pickle', 'wb') as handle:
    pickle.dump(player_table_big_5, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [165]:
# # Reading player_table pickle file

# with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_table.pickle', 'rb') as handle:
#     pickle_player_table = pickle.load(handle)

# pickle_player_table

In [184]:
# Reading Big 5 player_table pickle file

with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_table_big_5.pickle', 'rb') as handle:
    player_table_big_5 = pickle.load(handle)

player_table_big_5

['/en/players/774cf58b/matchlogs/2021-2022/summary/Max-Aarons-Match-Logs',
 '/en/players/f2bf1b0f/matchlogs/2021-2022/summary/Che-Adams-Match-Logs',
 '/en/players/9b398aea/matchlogs/2021-2022/summary/Rayan-Ait-Nouri-Match-Logs',
 '/en/players/a8c0acb7/matchlogs/2021-2022/summary/Kristoffer-Ajer-Match-Logs',
 '/en/players/eaeca114/matchlogs/2021-2022/summary/Nathan-Ake-Match-Logs',
 '/en/players/b827d5b3/matchlogs/2021-2022/summary/Marc-Albrighton-Match-Logs',
 '/en/players/77e84962/matchlogs/2021-2022/summary/Thiago-Alcantara-Match-Logs',
 '/en/players/cd1acf9d/matchlogs/2021-2022/summary/Trent-Alexander-Arnold-Match-Logs',
 '/en/players/7a2e46a8/matchlogs/2021-2022/summary/Alisson-Match-Logs',
 '/en/players/cc700722/matchlogs/2021-2022/summary/Allan-Match-Logs',
 '/en/players/cea4ee8f/matchlogs/2021-2022/summary/Dele-Alli-Match-Logs',
 '/en/players/862a1c15/matchlogs/2021-2022/summary/Miguel-Almiron-Match-Logs',
 '/en/players/f4290206/matchlogs/2021-2022/summary/Marcos-Alonso-Match-Lo

In [185]:
len(player_table_big_5)

81256

In [191]:
# Pull all player stats for all competitions

def get_players_all_competitions(player_list):
    
    player_urls = []

    for i in player_list:
        # player_urls.append('https://fbref.com/en/players/' + i.split('/')[3:4][0] + '/' + i.split('/')[7:][0].replace("-Match-Logs", ""))
        player_urls.append('https://fbref.com/en/players/' + i.split('/')[3:4][0] + '/all_comps/' 
                            + i.split('/')[7:][0].replace("-Match-Logs", "") + '/-Stats---All-Competitions')

    return list(set(player_urls))


player_all_competitions = get_players_all_competitions(player_table_big_5)
len(player_all_competitions)


19564

In [192]:
# Generate the match log urls for all players across all 

def get_player_match_logs(player_list_summary, line):
    
    res = requests.get(player_list_summary[line])
    soup = BeautifulSoup(res.text,'lxml')

    match_logs_list = []

    for i in soup.find_all('tbody'):
        for j in i.find_all('td', {'data-stat':'matches'}):
            if j.find('a') != None:
                if 'summary' in j.find('a')['href']:
                    match_logs_list.append(j.find('a')['href'])
                    
    return list(set(match_logs_list))

In [193]:

# Total length of player_all_competitions is 19,535
# List will be splitted in 5 

match_logs_list = []

# 1st batch 0:5000 - DONE
count = 0
for i in range(len(player_all_competitions[0:5000])):
    match_logs_list.extend(get_player_match_logs(player_all_competitions[0:5000], i))
    count += 1
    sys.stdout.write("\r{0} percent".format((count / len(player_all_competitions[0:5000])*100)))
    sys.stdout.flush()

100.0 percent

In [195]:
# export to personal drive as match_logs_list_1.csv - DONE

#Victor's Path
# pd.DataFrame(match_logs_list).to_csv('/Users/vruiz.CDS/Downloads/Capstone Files/match_logs_list_1.csv')

#Renzo's Path
pd.DataFrame(match_logs_list).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/match_logs_list_1.csv')

In [196]:
# 2nd batch 5000:10000 - PENDING

match_logs_list = []
count = 0

for i in range(len(player_all_competitions[5000:10000])):
    match_logs_list.extend(get_player_match_logs(player_all_competitions[5000:10000], i))
    count += 1
    sys.stdout.write("\r{0} percent".format((count / len(player_all_competitions[5000:10000])*100)))
    sys.stdout.flush()

100.0 percent

In [197]:
# export to personal drive as match_logs_list_2.csv - PENDING

pd.DataFrame(match_logs_list).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/match_logs_list_2.csv')

In [198]:
len(match_logs_list)

36773

In [199]:
# 3rd batch 10000:15000 - PENDING

match_logs_list = []
count = 0

for i in range(len(player_all_competitions[10000:15000])):
    match_logs_list.extend(get_player_match_logs(player_all_competitions[10000:15000], i))
    count += 1
    sys.stdout.write("\r{0} percent".format((count / len(player_all_competitions[10000:15000])*100)))
    sys.stdout.flush()

100.0 percent

In [200]:
# export to personal drive as match_logs_list_3.csv - PENDING

# Victor's Path
# pd.DataFrame(match_logs_list).to_csv('/Users/vruiz.CDS/Downloads/Capstone Files/match_logs_list_3.csv')

# Renzo's Path
pd.DataFrame(match_logs_list).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/match_logs_list_3.csv')

In [15]:
# 4th batch 15000:-1 - PENDING

match_logs_list = []
count = 0

for i in range(len(player_all_competitions[15000:-1])):
    match_logs_list.extend(get_player_match_logs(player_all_competitions[15000:-1], i))
    count += 1
    sys.stdout.write("\r{0} percent".format((count / len(player_all_competitions[15000:-1])*100)))
    sys.stdout.flush()

In [ ]:
# export to personal drive as match_logs_list_4.csv - PENDING

#Victor's Path
# pd.DataFrame(match_logs_list).to_csv('/Users/vruiz.CDS/Downloads/Capstone Files/match_logs_list_4.csv')

#Renzo's Path
pd.DataFrame(match_logs_list).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/match_logs_list_4.csv')